### This is for PRF1 ELANE and ILR7 

In [5]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import numpy as np
import os
import sys
import gzip
import itertools
import operator
import subprocess
import twobitreader
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pysam
import shutil

#from uditas.uditas_replace_helpers import *
#from uditas.uditas_guideseq import *

from guideseq_functions.guideseq_helpers_ElaneIL7Prf import *
from guideseq_functions.guideseq_ElaneIL7Prf import *



### Problem log

** makes sure you keep your I5 primers binding to the ODN sequence as this becomes "Read1 by the machine"
1. do 200 long read for Read1 

2.naming files and folder
2. longer site in front of primer binding
3. cut out amplicons that are 400-700 so I don't loose reads


In [6]:
#Directory

directory = '/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq3_ELANE_IL7R_PRF1'

print(directory)

/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq3_ELANE_IL7R_PRF1


### Discard Mispriming Reads
When you put a universal primer on the ends of everything, every mispriming event will amplify. An effect we normally don't deal with. I did nested PCR to reduce this. However 85% of the alignments in the UDITAS data I looked at seemed to be mispriming. They did all their blasting and analysis before removing mispriming. But to save computational power and remove error early on I will discard mispriming events. 
They discard these only for plasmid alignments analyze_alignments_plasmid for some reason which comes from the bam file.

### for guideseq read1 is where the ODN primer binds and so that is how you tell targeting. For the reverse it was binidng to the transposon which is random but then the seq primer binds right on it so the the read for read2 begins in the genomic sequence. the Rev primer could misprime but it shouldnt matter as it would then just bind and make smaller amplicons

In [7]:
#the amplicon info is related to the line on the csv file. It is indexed from 0.
#it is on my extera space so need to make sure that is mounted. (first thing to check if it throws and error)


# THIS IS WHAT RUNS THE PRIMER CHECK THROUGH ALL SAMPLES
#input the sample range or list below


results_df_all = pd.DataFrame()

results_folder = os.path.join(directory, 'results')
if not os.path.exists(results_folder):
    os.mkdir(results_folder)
results_file = os.path.join(directory, 'results','all_priming.xlsx')
    

#make results tree

for i in range(48):
        
    amplicon_info = get_csv_data(directory, i)

    #5primer is everything but AT, AT is for checking mispriming. The full sequence is the olgio from guideseq.
    #EVERYTHING IS CAPITAL
    guideseq3_seq = 'GTTTAATTGAGTTGTCATATGTTAATAACGGTAT'
    guideseq3_primeronly = 'GTTTAATTGAGTTGTCATATGTTAATAACGGT'
    #3primer is everything but the final AC. The AC is for checking mispriming
    guideseq5_seq = 'ATACCGTTATTAACATATGACAACTCAATTAAAC'
    guideseq5_primeronly = 'ATACCGTTATTAACATATGACAACTCAATTAA'

    direction = amplicon_info['Direction']

    if direction == 3:
        primer_seq_plus_downstream = guideseq3_seq
        primer_seq = guideseq3_primeronly
    elif direction == 5:
        primer_seq_plus_downstream = guideseq5_seq
        primer_seq = guideseq5_primeronly
    
    df_sample_results = correct_priming_guideseq(directory, amplicon_info, primer_seq, primer_seq_plus_downstream)
    results_df_all = results_df_all.append(df_sample_results, ignore_index=True)
    print('done with sample', i)

print(results_df_all)
results_df_all.to_excel(results_file)    
#print reults to excel

done with sample 0
done with sample 1
done with sample 2
done with sample 3
done with sample 4
done with sample 5
done with sample 6
done with sample 7
done with sample 8
done with sample 9
done with sample 10
done with sample 11
done with sample 12
done with sample 13
done with sample 14
done with sample 15
done with sample 16
done with sample 17
done with sample 18
done with sample 19
done with sample 20
done with sample 21
done with sample 22
done with sample 23
done with sample 24
done with sample 25
done with sample 26
done with sample 27
done with sample 28
done with sample 29
done with sample 30
done with sample 31
done with sample 32
done with sample 33
done with sample 34
done with sample 35
done with sample 36
done with sample 37
done with sample 38
done with sample 39
done with sample 40
done with sample 41
done with sample 42
done with sample 43
done with sample 44
done with sample 45
done with sample 46
done with sample 47
            sample_name    i7    i5  total_reads  

In [8]:
### TRIMMING ####
#need to trim off the end of the short reads. This is for amplicons that were too short and have the other side on them.

for i in range(48):
        
    amplicon_info = get_csv_data(directory, i)
    
    trim_guideseq(directory, amplicon_info)
    
    print('done with sample', i)


done with sample 0
done with sample 1
done with sample 2
done with sample 3
done with sample 4
done with sample 5
done with sample 6
done with sample 7
done with sample 8
done with sample 9
done with sample 10
done with sample 11
done with sample 12
done with sample 13
done with sample 14
done with sample 15
done with sample 16
done with sample 17
done with sample 18
done with sample 19
done with sample 20
done with sample 21
done with sample 22
done with sample 23
done with sample 24
done with sample 25
done with sample 26
done with sample 27
done with sample 28
done with sample 29
done with sample 30
done with sample 31
done with sample 32
done with sample 33
done with sample 34
done with sample 35
done with sample 36
done with sample 37
done with sample 38
done with sample 39
done with sample 40
done with sample 41
done with sample 42
done with sample 43
done with sample 44
done with sample 45
done with sample 46
done with sample 47


In [9]:
# BOWTIE2_INDEXES are needed for global alignments
#check in bash: > ECHO $GENOMES_2BIT
%env BOWTIE2_INDEXES=/home/eric/Data/Ref_Genomes

env: BOWTIE2_INDEXES=/home/eric/Data/Ref_Genomes




## Need to make an new bowtie2 index file that includes targeting for alignment agains the whole genome so it is all in one sheet together. 

### Other option would be to align it to the amplicon, extract unaligned files and then align to the genome but seems cleaner this way. 
#### Ideally every sequence is unique between the targeting vector and genome.


1. Build your fastas of interest and label .fa files.
    1. You need fasta of hg38 or reference genome. You can pull this from downloaded bowtie indexed sampels and then use the following command to turn the index into a fasta file: bowtie2-inspect hg38 > hg38.fa   
    2. Put all the fasta files in the same folder. Should also use the transfected plasmid
2. index the files with bowtie
    1. use the command bowtie2-build -f pE049,pe038_mc.fa,hg38.fa -p hg38_plus_targetvectorandplasmid
    2. this has the -p to make it take less ram in my case.
    3. In this case it adds the hg38 and the minicircle targeting file together
    4. I have a Intel® Core™ i7-5500U CPU @ 2.40GHz × 4 with 15.1 GiB ram and it required about 13.8 gigs of ram and 2 hours to do a hg38+small fasta index
    5. be sure to pay attention to the name of the new indexed file. "hg38_plus_targetvector" in example above. Add it to the sample_info.csv sheet. Under the tab 'genome_plus_targeting'.
    6. you can check it indexed correcctly: bowtie2-inspect -s hg38_plus_targetvector
    

    



## Now we blast end-to-end the correctly primed sequences.
We don't need to trim off anything. But we should remove the length of the ODN primer sequence before blasting for Read1

In [11]:
### This section does two things. It makes a bam file that is filtered for AS and MAPQ 
#   It quantifies these bam files and sorts them. Then it converts to a bed file.
#

#### Required Arguments
cpu = 12
min_MAPQ = 50
min_AS = -180

### WHAT TYPE OF ANALYSIS ARE WE RUNNING
#####type in 'on' or 'off'
local = 'off' #local is softclipped NEED TO MAKE THIS TO USE IT
global_align_branch = 'on' #global is end-to-end and this does the analysis
#this does the acutal alignment for end-to-end
do_align_global = 'on'
single_or_paired_reads = "paired"  #can be 'single' or 'paired' for the read alignment

#### WHAT SAMPLES TO RUN ####
#lines_to_run = [20]
lines_to_run = range(48)

#for exporting summary of data
results_df_all = pd.DataFrame()
results_file = os.path.join(directory, 'results','alignment_filtered_summary.xlsx')


for i in lines_to_run:
    
    amplicon_info = get_csv_data(directory, i)
    print('running row', i)
    print('the analysis is on Guideseq sample:', amplicon_info['name'])
    reference_assembly = amplicon_info['genome']
    sample_name = amplicon_info['name']
    
    if global_align_branch == 'on':
        if do_align_global == 'on':
            #####this aligns everything after the break to end-to-end keep_sam=1 means keep the sam file
            align_guideseq_end_to_end_genome_global(directory, amplicon_info, reference_assembly, single_or_paired_reads, cpu, keep_sam=1)

            
        #### Process the alignment files to make a bam file of only high quality reads
        
        #this filters for good alignments based on AS score and MapQ and exports a final indexed bam file.

        df_sample_results = guideseq_filtered_mapq_AS_primary(directory, 'global', amplicon_info, single_or_paired_reads)
        #this is a running dataframe of the clenaed up read summary
        results_df_all = results_df_all.append(df_sample_results, ignore_index=True)
        
        
        ### global alignment bed generation 
        N7 = amplicon_info['index_I1']
        N5 = amplicon_info['index_I2']
        print('-----  now making bed file for', sample_name, ' ',N7, ' ', N5 )
        
        
        ##### Need to fix this!!!!!!!!!!!! the names of the input are not right and mabye i need to sort things
        if single_or_paired_reads == "single":
            genome_global_bed_file = create_filename(directory, N7, N5, 'guideseq_global_bed_single')
            filtered_and_sorted_bam_file = create_filename(directory, N7, N5, 'filtered_and_sorted_genome_global_single')
        elif single_or_paired_reads == "paired":
            genome_global_bed_file = create_filename(directory, N7, N5, 'guideseq_global_bed_paired')
            filtered_and_sorted_bam_file = create_filename(directory, N7, N5, 'filtered_and_sorted_genome_global_paired')
    
        !bam2bed < {filtered_and_sorted_bam_file} > {genome_global_bed_file}
        bed_folder = os.path.join(directory, 'bed_files')

        if not os.path.exists(bed_folder):
            os.mkdir(bed_folder)
        
        file_location = create_filename(directory, N7, N5, 'bam_global')
        
        new_bed_name = os.path.join(file_location, sample_name + '_' + single_or_paired_reads + '.bed')

        os.rename(genome_global_bed_file, new_bed_name)

        shutil.copy(new_bed_name, bed_folder)
        
        
        

        print('made global bed file:', N7, '_', N5 )
        print('It is called',sample_name )
        print('doing the quantification of global alignments')

 
    
    ########### I don't use local so ignore this #############
    if local == 'on':
        #### this aligns everything in the local format wehre it can soft clip the ends keep_sam=1 means keep the sam file
        # need to make this section if I want to use it
        align_afterbreaks_genome_local(directory, 1, amplicon_info, assembly_plus_targetvector, lam_or_tn5='lam', keep_sam=0)
        
        #make bed file
        print('-----  now making bed file   ------')
        N7 = amplicon_info['index_I1']
        N5 = amplicon_info['index_I2']
        file_trimmed_genome_local_bed = create_filename(directory, N7, N5, 'break_trimmed_genome_local_bed')
        file_sorted_bam_genome_local = create_filename(directory, N7, N5, 'break_trimmed_filtered_and_sorted_bam_genome_local')

        !bam2bed < {file_sorted_bam_genome_local} > {file_trimmed_genome_local_bed}
        bed_folder = os.path.join(directory, 'bed_files')
        
        if not os.path.exists(bed_folder):
            os.mkdir(bed_folder)
        shutil.copy(file_trimmed_genome_local_bed, bed_folder)
    
        
        print('made local bed file:', N7, '_', N5 )

        # running overview of alignment quantification
        print('doing the quantification of local alignments')
        quantify_local = quantify_pipeline2_alignments(directory, 'local', amplicon_info, 'lam')
        print(quantify_local)

#export all the read summaries
results_df_all.to_excel(results_file)    


running row 0
the analysis is on Guideseq sample: 3'GSP-Ctrl-1
single por paired paired
    sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  3'GSP-Ctrl-1  N701  S502                 paired                 28484   

   filtered_alignments_count  
0                      11147  
-----  now making bed file for 3'GSP-Ctrl-1   N701   S502
made global bed file: N701 _ S502
It is called 3'GSP-Ctrl-1
doing the quantification of global alignments
running row 1
the analysis is on Guideseq sample: 3'GSP-Ctrl-2
single por paired paired
    sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  3'GSP-Ctrl-2  N702  S502                 paired                 91679   

   filtered_alignments_count  
0                      37364  
-----  now making bed file for 3'GSP-Ctrl-2   N702   S502
made global bed file: N702 _ S502
It is called 3'GSP-Ctrl-2
doing the quantification of global alignments
running row 2
the analysis is on Guideseq sample: 3'GSP-Ctrl-3
sin

running row 17
the analysis is on Guideseq sample: 3'GSP-IL7R-Cas9n-3
single por paired paired
          sample_name    i7    i5 single_or_paired_reads  \
0  3'GSP-IL7R-Cas9n-3  N706  S503                 paired   

   all_alignments_count  filtered_alignments_count  
0                 34217                      23174  
-----  now making bed file for 3'GSP-IL7R-Cas9n-3   N706   S503
made global bed file: N706 _ S503
It is called 3'GSP-IL7R-Cas9n-3
doing the quantification of global alignments
running row 18
the analysis is on Guideseq sample: 3'GSP-PRF1-Cas9-1
single por paired paired
         sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  3'GSP-PRF1-Cas9-1  N707  S503                 paired                130455   

   filtered_alignments_count  
0                      58151  
-----  now making bed file for 3'GSP-PRF1-Cas9-1   N707   S503
made global bed file: N707 _ S503
It is called 3'GSP-PRF1-Cas9-1
doing the quantification of global alignments
running r

single por paired paired
           sample_name    i7    i5 single_or_paired_reads  \
0  5'GSP-ELANE-Cas9n-2  N714  S505                 paired   

   all_alignments_count  filtered_alignments_count  
0                 40221                      20411  
-----  now making bed file for 5'GSP-ELANE-Cas9n-2   N714   S505
made global bed file: N714 _ S505
It is called 5'GSP-ELANE-Cas9n-2
doing the quantification of global alignments
running row 35
the analysis is on Guideseq sample: 5'GSP-ELANE-Cas9n-3
single por paired paired
           sample_name    i7    i5 single_or_paired_reads  \
0  5'GSP-ELANE-Cas9n-3  N715  S505                 paired   

   all_alignments_count  filtered_alignments_count  
0                 41774                      32051  
-----  now making bed file for 5'GSP-ELANE-Cas9n-3   N715   S505
made global bed file: N715 _ S505
It is called 5'GSP-ELANE-Cas9n-3
doing the quantification of global alignments
running row 36
the analysis is on Guideseq sample: 5'GSP-IL7R-Cas

In [72]:
############ Use CIRPSR Gold and look at overlaps

single_or_paired_reads = "paired"  #can be 'single' or 'paired' for the read alignment
windowsize = 500

#### WHAT SAMPLES TO RUN ####
lines_to_run = [1]
#lines_to_run = range(30)


#off_target_sites 
crispr_gold_file = '/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/CrispRgold_predictions.xlsx'
df_targets = pd.read_excel(crispr_gold_file, index_col=0)

# new data frame with split guide location. CRISPRGOLD puts it all in at once
new = df_targets["Position"].str.split(":", expand = True) 
df_targets["chr"]= new[0] 
df_targets["position_start"]= new[1] 
df_targets["position_end"]= new[2] 
df_targets.drop(columns =["Position"], inplace = True) 
df_targets['position_start'] = df_targets['position_start'].astype(int)
df_targets['begin_site_range'] = df_targets['position_start'] - windowsize
df_targets['end_site_range'] = df_targets['position_start'] + windowsize

print(df_targets.dtypes)

#print(df_targets)


for i in lines_to_run:
    
    amplicon_info = get_csv_data(directory, i)
    name = amplicon_info['name']
    print('running row', i)
    print('the analysis is on Guideseq sample:', name)
    print('index',)
    
    #import bed file
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    print('index', N7, N5)



    
    print(results_file)

    file_location = create_filename(directory, N7, N5, 'bam_global')   
    print(file_location)
    sample_name = amplicon_info['name']
    bedfile = os.path.join(file_location, sample_name + '_' + single_or_paired_reads + '.bed')
    print(bedfile)
    
    #needs to add header names to specify the number of columns so that 
    
    header_names = ['chr', 'chromStart', 'chromEnd', 'name', 'MapQ', 'strand', 'flag', 'CIGAR', 'paired_name', 'read1start', 'paired_amplicon_size', 'sequence', 'quality_score','a','b','c','d','e','f','g','h','i']

#    df_bed = pd.read_csv(bedfile, sep='\t', comment='t', header=None)
    df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = header_names)

    
    
    #header = ['chr', 'chromStart', 'chromEnd', 'name', 'MapQ', 'strand', 'flag', 'CIGAR', 'paired_name', 'read1start', 'paired_amplicon_size', 'sequence', 'quality_score','a','b','c','d','e','f','g','h','i']
    #df_bed.columns = header[:len(df_bed.columns)]
    
    print(df_bed.dtypes) 
    
    read_hits = []
    count = 0
    for i,g in itertools.izip(df_bed.chromStart, df_bed.chr):
        bed_alignment = i
        chromosome = g
        for a,b,c,d in itertools.izip(df_targets.listnumber2, df_targets.begin_site_range, df_targets.end_site_range, df_targets.chr): 
            counter_A = 0
            if bed_alignment in range(b, c) and chromosome == d:
                if counter = 0:
                    print(b,c,'chromosome', d, ' guide:', a)
                    read_hits.append(a)
                    counter_A += 1
                    print(b,c,'chromosome', d, ' guide:', a, 'for read starting at :', i)
        count += 1 
        if count >30:
            break
    results_df = pd.DataFrame({'sample_name': [name],
                               'i7': [N7],
                               'i5': [N5],
                               'list_of_read_guide_matches' : [read_hits],
                               },
                                  columns=['sample_name',
                                           'i7',
                                           'i5',
                                           'list_of_read_guide_matches'])            
    
    
    
    results_file = os.path.join(directory, 'results',N7 + '_' + N5 + '_'+ name +'_guide_count.xlsx')
    
    results_df.to_excel(results_file)    

    print('list of read hits:',read_hits)


listnumber2                 object
Sequence                    object
Name                        object
Top off-target sites        object
Strand                      object
MutDist                      int64
Risk                        object
Annotation (if relevant)    object
chr                         object
position_start               int64
position_end                object
begin_site_range             int64
end_site_range               int64
dtype: object
running row 1
the analysis is on Guideseq sample: 2-1-3'GSP-TD
index
index N702 S502
/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/results/N702_S502_2-1-3'GSP-TD_guide_count.xlsx
/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/N702_S502/bam_genome_global_files
/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/N702_S502/bam_genome_global_files/2-1-3'GSP-TD_paired.bed
chr                     object
chromStart               int64
chromEnd                 int64
name                    object
MapQ                     int64
stra